---
title: SpatioTemporal Field - T2M - Non-Stationary + GP
subject: AEMET T2M
short_title: 3 - Station Predictions
authors:
  - name: J. Emmanuel Johnson
    affiliations:
      - CSIC
      - UCM
      - IGEO
    orcid: 0000-0002-6739-0053
    email: juanjohn@ucm.es
license: CC-BY-4.0
keywords: notation
---

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" # first gpu
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'FALSE'

import jax
jax.config.update('jax_platform_name', 'cpu')

import numpyro
import multiprocessing

num_devices = multiprocessing.cpu_count()
numpyro.set_platform("cpu")
numpyro.set_host_device_count(num_devices)
jax.config.update("jax_enable_x64", True)

In [2]:
import autoroot
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import pint_xarray
import arviz as az

from st_evt.viz import plot_histogram, plot_density
from st_evt._src.modules.models.aemet import utils_station
from omegaconf import OmegaConf

import jax
import jax.random as jrandom
import jax.numpy as jnp
import pandas as pd

rng_key = jrandom.PRNGKey(123)

from numpyro.infer import Predictive
import arviz as az

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
import xarray as xr
import regionmask

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter
import seaborn as sns
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)

from loguru import logger

# num_devices = 5
# numpyro.set_host_device_count(num_devices)


%matplotlib inline
%load_ext autoreload
%autoreload 2

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


## Paths

In [3]:
results_root_path = "/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/"
results_file_name = "results.nc"
results_data_path = Path(results_root_path).joinpath(results_file_name)

figures_path = Path(results_root_path).joinpath("figures/stations/posterior")

## Data

In [4]:
DATA_URL = autoroot.root.joinpath("data/ml_ready/aemet/t2max_stations_bm_summer.zarr")
variable = "t2max"
covariate = "gmst"
spatial_dim_name = "station_id"


# LOAD DATA
with xr.open_dataset(DATA_URL, engine="zarr") as f:
    ds_bm = f.load()
    # ds_bm = ds_bm.where(ds_bm.red_feten_mask == 1, drop=True)

### Likelihood Statistics

There are some useful statistics that we can use to evaluate how well our model does.

$$
\begin{aligned}
\text{ELPD WAIC}: && && \\
\text{ELPD WAIC SE}: && && \\
\text{P-Value WAIC}: && && \\
\end{aligned}
$$

In [5]:
variable = "t2max"

## Station Selection

### a - Predetermined Station

In [6]:
candidate_station = '3129A' # '1391' # 

In [7]:
figures_path = figures_path.joinpath(f"{candidate_station}")
figures_path.mkdir(parents=True, exist_ok=True)

### MCMC Results

In [8]:
az_ds = az.from_netcdf(str(results_data_path))
az_ds_station_pred = az_ds.predictions.sel(station_id = candidate_station)
# num_samples = 5_000
# az_ds_station_pred = az.extract(az_ds, group="predictions", num_samples=num_samples).sel(station_id = candidate_station)
y_data = ds_bm.sel(station_id = candidate_station)[variable]
ds_station = ds_bm.sel(station_id = candidate_station)

### EDA Stuff

In [9]:
from st_evt.viz import plot_scatter_ts, plot_histogram, plot_density

In [10]:
utils_station.plot_eda(
    da=ds_station[variable].squeeze(),
    variable_label="2m Max Temperature [°C]",
    # figures_path="./", 
    figures_path=figures_path, 
    figure_dpi=300,
)

2025-01-09 20:18:20.054 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_eda:114 - Plotting BM Data Time Series...
2025-01-09 20:18:20.478 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_eda:130 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/eda/ts_bm_data.pdf
2025-01-09 20:18:20.479 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_eda:132 - Plotting BM Data Histogram...
2025-01-09 20:18:20.621 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_eda:146 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/eda/hist_bm_data.pdf
2025-01-09 20:18:20.622 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_eda:148 - Plotting BM Data Density...
2025-01-09 20:18:20.858 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_eda:162 - Saved Figure

:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](eda/ts_bm_data.png)

![My vacation pics! 🏝](eda/hist_bm_data.png)

![My vacation pics! 🏝](eda/density_bm_data.png)


Some pictures of fruit and the ocean!
:::

## Posterior Calculations

## Model Inspection

### Trace Plot

In [11]:
variables = [
    "concentration",
    "scale",
    "location_slope",
    "location_intercept",
    ]

utils_station.plot_model_params_critique(
    ds=az_ds_station_pred,
    variables=variables,
    # figures_path="./", 
    figures_path=figures_path, 
    
)

2025-01-09 20:18:20.935 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:39 - Plotting Parameter Traces...
2025-01-09 20:18:21.474 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:53 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/params/trace.pdf
2025-01-09 20:18:21.475 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:55 - Plotting Parameter Jonts...
2025-01-09 20:18:22.532 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:76 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/params/joint.pdf
2025-01-09 20:18:22.533 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:77 - Plotting AutoCorrelation...
2025-01-09 20:18:22.802 | DEBUG    | 

## Regression Plot

In [12]:
observations = ds_station[variable].squeeze()

# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

az_ds_quantiles = az_ds_station_pred.quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
az_ds_quantiles = az_ds_quantiles.assign_coords({covariate: az_ds_quantiles[covariate]})
az_ds_quantiles = az_ds_quantiles.swap_dims({"time": "gmst"})

# LOCATION PARAMETER
locations = az_ds_station_pred["location"].sel(variable=variable).quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
locations = locations.swap_dims({"time": "gmst"})
locations = locations.assign_coords({covariate: az_ds_station_pred[covariate].sel(covariate=covariate).values})

# SCALE PARAMETER
scales = az_ds_station_pred["scale"].sel(variable=variable).quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
# scales = scales.swap_dims({"time": "gmst"})

# RETURN LEVEL
return_level_100 = az_ds_station_pred["return_level_100"].sel(variable=variable).quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
# return_level_100 = return_level_100.swap_dims({"time": "gmst"})

In [13]:
utils_station.plot_regression_prediction(
    ds_quantiles=az_ds_quantiles,
    observations=observations,
    # figures_path="./", 
    figures_path=figures_path, 
    covariate=covariate,
    figure_dpi=300,
    y_label="2m Max Temperature, $R_a$ [°C]",
    covariate_label="Global Mean Surface Temperature Anomaly [°C]",
    location_only=True
)

utils_station.plot_regression_prediction(
    ds_quantiles=az_ds_quantiles,
    observations=observations,
    # figures_path="./", 
    figures_path=figures_path, 
    covariate=covariate,
    figure_dpi=300,
    y_label="2m Max Temperature, $R_a$ [°C]",
    covariate_label="Global Mean Surface Temperature Anomaly [°C]",
    location_only=False
)

2025-01-09 20:18:25.342 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_regression_prediction:1044 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/regression/regression_pred_location.png
2025-01-09 20:18:25.659 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_regression_prediction:1044 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/regression/regression_pred.png


## Return Levels

#### Empirical Return Levels

In [14]:
# select clean data
y_clean = ds_station[variable].squeeze()
y_clean = y_clean.assign_coords({covariate: ds_bm[covariate].values})


In [15]:
# select clean data
y_clean = ds_station[variable].squeeze()
y_clean = y_clean.assign_coords({covariate: ds_bm[covariate].values})

# calculate return period
y_clean = utils_station.calculate_empirical_return_level_gevd_ds(y_clean, covariate=covariate)
y_clean = y_clean.swap_dims({variable: "gmst"})

# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

# Calculate Quantiles
rl_model_quantiles = az_ds_station_pred["return_level"].quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"])
rl_model_quantiles = rl_model_quantiles.sel(variable=variable)
rl_model_quantiles = rl_model_quantiles.assign_coords({covariate: az_ds_station_pred[covariate].sel(covariate=covariate)})
rl_model_quantiles = rl_model_quantiles.swap_dims({"time": "gmst"})
# rl_model_quantiles = rl_model_quantiles.assign_coords({covariate: ds_bm[covariate].values})

2025-01-09 20:18:25.788 | INFO     | st_evt._src.modules.models.aemet.utils_station:calculate_empirical_return_level_gevd_ds:354 - Calculating Return Level...
2025-01-09 20:18:25.790 | INFO     | st_evt._src.modules.models.aemet.utils_station:calculate_empirical_return_level_gevd_ds:362 - Swapping Dims...


### Viz - Return Level 

In [16]:
# fig, ax = model_eval_station.plot_return_periods_ds(
#     rl_model_quantiles=rl_model_quantiles,
#     y=y_clean,
#     covariate=covariate,
#     y_label="2m Max Temperature, $R_a$ [°C]"
# )
# plt.show()

utils_station.plot_return_periods_gmst_ds(
    rl_model_quantiles=rl_model_quantiles,
    y=y_clean,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    y_label="2m Max Temperature, $R_a$ [°C]"
)

2025-01-09 20:18:26.275 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:476 - Getting Appropriate Periods...
2025-01-09 20:18:26.275 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:478 - Intialize Returns...
2025-01-09 20:18:26.276 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:482 - Creating Data structures...
2025-01-09 20:18:26.277 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:505 - Plotting...
2025-01-09 20:18:27.171 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:521 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/returns/returns_gmst_prob_posterior_vs_empirical.png


### Viz - 100-Year Return Period

In [17]:
# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

# Calculate Quantiles
rl_model_quantiles = az_ds_station_pred["return_level_100"].sel(variable=variable)
rl_model_quantiles = rl_model_quantiles.assign_coords({covariate: az_ds_station_pred[covariate].sel(covariate=covariate)})
rl_model_quantiles = rl_model_quantiles.swap_dims({"time": "gmst"})

In [18]:
# fig, ax = plot_return_periods_100_ds(
#     rl_model_quantiles=rl_model_quantiles,
#     covariate=covariate,
#     x_label="2m Max Temperature, $R_a$ [°C]"
# )
# plt.show()

utils_station.plot_return_periods_100_gmst_ds(
    rl_model=rl_model_quantiles,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    x_label=r"2m Max Temperature, $R_{100}$ [°C]"
)

2025-01-09 20:18:27.370 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:632 - Getting Appropriate Periods...
2025-01-09 20:18:27.371 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:634 - Intialize Returns...
2025-01-09 20:18:27.371 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:638 - Creating Data structures...
2025-01-09 20:18:27.373 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:661 - Plotting...
2025-01-09 20:18:27.708 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:677 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationary_gp_lap_demo/figures/stations/posterior/3129A/returns/returns_100years_gmst_density.pdf


### Viz - 100-Year Return Period Difference

In [19]:
# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

# Calculate Quantiles
rl_model = az_ds_station_pred["return_level_100"].sel(variable=variable)
rl_model = rl_model.assign_coords({covariate: az_ds_station_pred[covariate].sel(covariate=covariate)})
rl_model = rl_model.swap_dims({"time": "gmst"})
rl_model

<xarray.DataArray 'return_level_100' (chain: 1, gmst: 50, draw: 1000)> Size: 400kB
array([[[40.52708601, 39.57461568, 40.89468659, ..., 43.62445081,
         41.48789793, 40.42645388],
        [40.61124299, 39.65531574, 40.98677185, ..., 43.71144218,
         41.56882135, 40.51001844],
        [40.69539997, 39.73601581, 41.07885712, ..., 43.79843355,
         41.64974477, 40.593583  ],
        ...,
        [44.48246405, 43.36751871, 45.22269408, ..., 47.71304522,
         45.29129874, 44.35398822],
        [44.56662103, 43.44821877, 45.31477934, ..., 47.80003659,
         45.37222216, 44.43755279],
        [44.65077801, 43.52891884, 45.40686461, ..., 47.88702796,
         45.45314559, 44.52111735]]])
Coordinates:
  * chain           (chain) int64 8B 0
  * draw            (draw) int64 8kB 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
    time            (gmst) int64 400B 0 1 2 3 4 5 6 7 ... 43 44 45 46 47 48 49
    variable        <U5 20B 't2max'
    alt             float32 4B 631.0
    lat             float32 4B 40.46
    lon             float32 4B -3.581
    red_feten_mask  uint8 1B 1
    station_id      <U5 20B '3129A'
    station_name    <U50 200B 'madrid,barajas rs'
    covariate       <U4 16B 'gmst'
  * gmst            (gmst) float64 400B 0.0 0.05102 0.102 ... 2.398 2.449 2.5

In [20]:
utils_station.plot_return_periods_100_difference_gmst_ds(
    rl_model=rl_model,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    x_label="2m Max Temperature, $\Delta R_{100}$ [°C]",
    units="[°C]",
)

2025-01-09 20:18:27.920 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:807 - Getting Appropriate Periods...
2025-01-09 20:18:27.921 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:809 - Calculating Difference...
2025-01-09 20:18:27.924 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:817 - Intialize Returns...
2025-01-09 20:18:27.926 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:820 - Creating Data structures...
2025-01-09 20:18:27.926 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:840 - Plotting...
2025-01-09 20:18:28.184 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:856 - Saved Figure:
/home/juanjohn/pool_projects/scratch/stevt_tutorial/models/nonstationa

In [21]:
# fig, ax = plot_return_periods_100_difference_ds(
#     rl_model=rl_model,
#     covariate=covariate,
#     x_label="2m Max Temperature, $R_a$ [°C]",
#     units="[°C]",
#     color="black"
# )
# plt.show()

utils_station.plot_return_periods_100_difference_prct_gmst_ds(
    rl_model=rl_model,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    x_label="2m Max Temperature, $\Delta R_{100}$ [%]",
)

2025-01-09 20:18:28.224 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_prct_gmst_ds:869 - Getting Appropriate Periods...
2025-01-09 20:18:28.225 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_prct_gmst_ds:871 - Calculating Difference...
2025-01-09 20:18:28.231 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_prct_gmst_ds:879 - Intialize Returns...
2025-01-09 20:18:28.231 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_prct_gmst_ds:882 - Creating Data structures...
2025-01-09 20:18:28.232 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_prct_gmst_ds:902 - Plotting...
2025-01-09 20:18:28.518 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_prct_gmst_ds:918 - Saved Figure:
/home/juanjohn/pool_projects/scratch/ste